# Splitting Data into Test-Train-Validation Sets

In [16]:
import csv
import argparse
import pandas as pd
import numpy as np
import os

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

In [17]:
batting_data_path = 'bsb_ref.csv'
# INFO:
# 101,332 Players with up to 20 features each (exluding year, including team)
# if metric not reported for player, set to 0.0 by default

df = pd.read_csv(batting_data_path, index_col = 0)
df = df.fillna(0)

In [18]:
df_recent_players = df[df.year_ID >= 1975] #48k players
team_set = set(df_recent_players.team_ID)
df_recent_players

#  NOTE: Potentially create one-hot encoding of teams...
## I chose to leave it off for now because I think it may
## create Data leakage in our model


age    mlb_ID  player_ID  year_ID team_ID  stint_ID lg_ID  \
name_common                                                                 
David Aardsma  22.0  430911.0  aardsda01     2004     SFG         1    NL   
David Aardsma  24.0  430911.0  aardsda01     2006     CHC         1    NL   
David Aardsma  25.0  430911.0  aardsda01     2007     CHW         1    AL   
David Aardsma  26.0  430911.0  aardsda01     2008     BOS         1    AL   
David Aardsma  27.0  430911.0  aardsda01     2009     SEA         1    AL   
David Aardsma  28.0  430911.0  aardsda01     2010     SEA         1    AL   
David Aardsma  30.0  430911.0  aardsda01     2012     NYY         1    AL   
David Aardsma  31.0  430911.0  aardsda01     2013     NYM         1    NL   
David Aardsma  33.0  430911.0  aardsda01     2015     ATL         1    NL   
Hank Aaron     41.0  110001.0  aaronha01     1975     MIL         1    AL   
Hank Aaron     42.0  110001.0  aaronha01     1976     MIL         1    AL   
Don Aase       22.0  110003.0   aasedo01     1977     BOS         1    AL   
Don Aase       23.0  110003.0   aasedo01     1978     CAL         1    AL   
Don Aase       24.0  110003.0   aasedo01     1979     CAL         1    AL   
Don Aase       25.0  110003.0   aasedo01     1980     CAL         1    AL   
Don Aase       26.0  110003.0   aasedo01     1981     CAL         1    AL   
Don Aase       27.0  110003.0   aasedo01     1982     CAL         1    AL   
Don Aase       29.0  110003.0   aasedo01     1984     CAL         1    AL   
Don Aase       30.0  110003.0   aasedo01     1985     BAL         1    AL   
Don Aase       31.0  110003.0   aasedo01     1986     BAL         1    AL   
Don Aase       32.0  110003.0   aasedo01     1987     BAL         1    AL   
Don Aase       33.0  110003.0   aasedo01     1988     BAL         1    AL   
Don Aase       34.0  110003.0   aasedo01     1989     NYM         1    NL   
Don Aase       35.0  110003.0   aasedo01     1990     LAD         1    NL   
Andy Abad      28.0  407577.0   abadan01     2001     OAK         1    AL   
Andy Abad      30.0  407577.0   abadan01     2003     BOS         1    AL   
Andy Abad      33.0  407577.0   abadan01     2006     CIN         1    NL   
Fernando Abad  24.0  472551.0   abadfe01     2010     HOU         1    NL   
Fernando Abad  25.0  472551.0   abadfe01     2011     HOU         1    NL   
Fernando Abad  26.0  472551.0   abadfe01     2012     HOU         1    NL   
...             ...       ...        ...      ...     ...       ...   ...   
Julio Zuleta   26.0  275936.0  zuletju01     2001     CHC         1    NL   
Joel Zumaya    21.0  451491.0  zumayjo01     2006     DET         1    AL   
Joel Zumaya    22.0  451491.0  zumayjo01     2007     DET         1    AL   
Joel Zumaya    23.0  451491.0  zumayjo01     2008     DET         1    AL   
Joel Zumaya    24.0  451491.0  zumayjo01     2009     DET         1    AL   
Joel Zumaya    25.0  451491.0  zumayjo01     2010     DET         1    AL   
Mike Zunino    22.0  572287.0  zuninmi01     2013     SEA         1    AL   
Mike Zunino    23.0  572287.0  zuninmi01     2014     SEA         1    AL   
Mike Zunino    24.0  572287.0  zuninmi01     2015     SEA         1    AL   
Mike Zunino    25.0  572287.0  zuninmi01     2016     SEA         1    AL   
Mike Zunino    26.0  572287.0  zuninmi01     2017     SEA         1    AL   
Mike Zunino    27.0  572287.0  zuninmi01     2018     SEA         1    AL   
Mike Zunino    28.0  572287.0  zuninmi01     2019     TBR         1    AL   
Bob Zupcic     24.0  124787.0  zupcibo01     1991     BOS         1    AL   
Bob Zupcic     25.0  124787.0  zupcibo01     1992     BOS         1    AL   
Bob Zupcic     26.0  124787.0  zupcibo01     1993     BOS         1    AL   
Bob Zupcic     27.0  124787.0  zupcibo01     1994     BOS         1    AL   
Bob Zupcic     27.0  124787.0  zupcibo01     1994     CHW         2    AL   
Paul Zuvella   23.0  124789.0  zuvelpa01     1982     ATL         1    NL   
Paul Zuvella

In [19]:
#  Randomly shuffle by player year
## NOTE: This may be an inaccurate assumption to make.
## We're implicitly assuming that training on players from 1975
## won't impact prediction accuracy for modern day players.

## If this is an incorrect assumption, consider data augmentation,
## taking into account differences between current-day MLB players

df_recent_players = df_recent_players.sample(frac=1) #randomly permute by year
player_ids = [df.iloc[:,0:0]]
df_recent_players
# player_ids = set(df_recent_players.playerID) #List of all player ids

# len(player_ids)

age    mlb_ID  player_ID  year_ID team_ID  stint_ID  \
name_common                                                                 
David Murphy         29.0  461815.0  murphda07     2011     TEX         1   
Jesse Hahn           25.0  534910.0   hahnje01     2015     OAK         1   
Steve Kline          29.0  117157.0  klinest02     2002     STL         1   
Domingo Ramos        25.0  120908.0  ramosdo01     1983     SEA         1   
Brian McRae          25.0  118871.0  mcraebr01     1993     KCR         1   
Bert Blyleven        25.0  111126.0  blylebe01     1976     MIN         1   
Matt Grott           27.0  115199.0  grottma01     1995     CIN         1   
Johnnie LeMaster     30.0  117665.0  lemasjo01     1984     SFG         1   
Rick Aguilera        28.0  110090.0  aguilri01     1990     MIN         1   
Rich Hinton          29.0  115975.0  hintori01     1976     CIN         1   
Ryan McGuire         25.0  118736.0  mcguiry01     1997     MON         1   
Mark Gubicza         30.0  115217.0  gubicma01     1993     KCR         1   
Ken Landreaux        25.0  117454.0  landrke01     1980     MIN         1   
Jeff Pierce          26.0  120557.0  piercje01     1995     BOS         1   
Jose Gonzalez        20.0  114931.0  gonzajo02     1985     LAD         1   
Max Stassi           27.0  545358.0  stassma01     2018     HOU         1   
Tom Burgmeier        36.0  111707.0  burgmto01     1980     BOS         1   
Kevin Orie           25.0  120047.0   orieke01     1998     CHC         1   
Ben Oglivie          31.0  119954.0  oglivbe01     1980     MIL         1   
Pete Dalena          29.0  113004.0  dalenpe01     1989     CLE         1   
Shane Andrews        26.0  110262.0  andresh01     1998     MON         1   
Dusty Baker          28.0  110481.0  bakerdu01     1977     LAD         1   
Steve Fireovid       27.0  114143.0  fireost01     1984     PHI         1   
Chris George         24.0  150203.0  georgch02     2004     KCR         1   
Ryan Perry           24.0  543634.0  perryry01     2011     DET         1   
Benny Ayala          28.0  110401.0  ayalabe01     1979     BAL         1   
Joe Wieland          25.0  543921.0  wielajo01     2015     LAD         1   
Wilson Betemit       26.0  400140.0  betemwi01     2008     NYY         1   
Scott Proctor        34.0  429984.0  proctsc01     2011     ATL         1   
Mike Schmidt         35.0  121836.0  schmimi01     1985     PHI         1   
...                   ...       ...        ...      ...     ...       ...   
Curt Schilling       40.0  121811.0  schilcu01     2007     BOS         1   
Gary Allenson        29.0  110164.0  allenga01     1984     BOS         1   
Felipe Lopez         22.0  150479.0  lopezfe01     2002     TOR         1   
Alberto Reyes        37.0  121099.0  reyesal01     2007     TBD         1   
Alejandro Pena       37.0  120369.0   penaal01     1996     FLA         1   
Brian Burres         25.0  434638.0  burrebr01     2006     BAL         1   
Chad Paronto         31.0  406301.0  paronch01     2007     ATL         1   
Asher Wojciechowski  26.0  592879.0  wojcias01     2015     HOU         1   
Ryan Jensen          25.0  276528.0  jensery01     2001     SFG         1   
Bob Robertson        32.0  121296.0  roberbo01     1979     TOR         1   
Austin Romine        29.0  519222.0  rominau01     2018     NYY         1   
Wilfredo Tovar       22.0  541600.0  tovarwi01     2014     NYM         1   
Freddie Freeman      25.0  518692.0  freemfr01     2015     ATL         1   
Tim Worrell          29.0  124580.0  worreti01     1997     SDP         1   
A.J. Morris          29.0  571963.0  morriaj01     2016     CIN         1   
Daniel Descalso      29.0  518614.0  descada01     2016     COL         1   
Jack Clark           36.0  112326.0  clarkja01     1992     BOS         1   
David Justice        25.0  116798.0  justida01     1991     ATL         1   
Vinny Castilla       29.0  112106.0  castivi02     1997     COL         1   
Kevin Morton      

In [25]:
x_data = df_recent_players[df_recent_players.columns[7:9]]
y_data = df_recent_players['OPS_plus']
print(x_data)

                        PA    G
name_common                    
David Murphy         440.0  120
Jesse Hahn             3.0    1
Steve Kline            1.0   64
Domingo Ramos        136.0   53
Brian McRae          685.0  153
Bert Blyleven          0.0    0
Matt Grott             0.0    2
Johnnie LeMaster     493.0  132
Rick Aguilera          0.0    1
Rich Hinton            1.0   12
Ryan McGuire         222.0   84
Mark Gubicza           0.0    0
Ken Landreaux        536.0  129
Jeff Pierce            0.0    0
Jose Gonzalez         12.0   23
Max Stassi           250.0   88
Tom Burgmeier          0.0    3
Kevin Orie           230.0   64
Ben Oglivie          660.0  156
Pete Dalena            7.0    5
Shane Andrews        559.0  150
Dusty Baker          604.0  153
Steve Fireovid         0.0    6
Chris George           3.0    1
Ryan Perry             0.0    1
Benny Ayala           94.0   42
Joe Wieland            2.0    1
Wilson Betemit       198.0   87
Scott Proctor          2.0   28
Mike Sch

In [26]:
reg = LinearRegression().fit(x_data, y_data)
print(reg.score(x_data, y_data))

0.2725445040216048
